# Feature selection and feature engineering

This isn't based on a news article, exactly, it's from a paper. You can see the paper in `/data/`.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)
%matplotlib inline

# Reading in our data

## Every single person in a crash

We'll start by reading in the list of people who were involved in an accident.

**Call this dataframe `people`.**

In [2]:
people = pd.read_csv("data/combined-person-data.csv")

C:\Users\Maximilian\AppData\Local\Programs\Python\Python37-32\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5,15,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


How often did each severity of injury show up? (e.g. not injured, non-incapacitating injury, etc)

In [3]:
people['INJ_SEVER_CODE'] = people.INJ_SEVER_CODE.astype(str)

In [4]:
people['INJ_SEVER_CODE'] = people.INJ_SEVER_CODE.str.replace(r"1","No Injury")
people['INJ_SEVER_CODE'] = people.INJ_SEVER_CODE.str.replace(r"2","Non-incapacitating Injury")
people['INJ_SEVER_CODE'] = people.INJ_SEVER_CODE.str.replace(r"3","Possible Incapacitating Injury")
people['INJ_SEVER_CODE'] = people.INJ_SEVER_CODE.str.replace(r"4","Incapacitating/Disabled Injury")
people['INJ_SEVER_CODE'] = people.INJ_SEVER_CODE.str.replace(r"5","Fatal Injury")

In [5]:
people.INJ_SEVER_CODE.value_counts()

No Injury                         716806
Non-incapacitating Injury          82642
Possible Incapacitating Injury     76801
Incapacitating/Disabled Injury     10353
Fatal Injury                        1681
Name: INJ_SEVER_CODE, dtype: int64

We're only interested in fatalities, so let's create a new `had_fatality` column for when people received a fatal injury.

**Confirm there were 1681 people with fatal injuries.**

In [6]:
people['had_fatality'] = people['INJ_SEVER_CODE'].str.replace(r"Fatal Injury", "1")
people['had_fatality'] = people.had_fatality.str.replace(r"No Injury", "0")
people['had_fatality'] = people.had_fatality.str.replace(r"Non-incapacitating Injury", "0" )
people['had_fatality'] = people.had_fatality.str.replace(r"Possible Incapacitating Injury", "0" )
people['had_fatality'] = people.had_fatality.str.replace(r"Incapacitating/Disabled Injury", "0" )
people['had_fatality'] = people.had_fatality.str.replace(r"Fatal Injury", "0")

In [7]:
people.had_fatality.value_counts()

0    886602
1      1681
Name: had_fatality, dtype: int64

## Working on Features

### Starting our analysis

We're going to run a regression on the impact of being **male vs female on crash fatalities**. Prepare a dataframe called `train_df` with the appropriate information in it.

* **Tip:** What column(s) are your input, and what is your output? Aka independent and dependent variables
* **Tip:** You'll need to convert your input column into something numeric, I suggest using `.replace`
* **Tip:** We aren't interested in the "Unknown" sex - either filtering or `np.nan` + `.dropna()` might be useful ways to get rid of those columns

In [13]:
people.SEX_CODE.isna().sum()

0

In [14]:
people.shape

(815827, 29)

In [15]:
people.SEX_CODE.value_counts()

M    460973
F    354854
Name: SEX_CODE, dtype: int64

In [11]:
people = people.dropna(subset=['SEX_CODE'])

In [12]:
people = people[people['SEX_CODE'] != 'U']

In [19]:
people.head(2)

,AIRBAG_DEPLOYED,ALCOHOL_TESTTYPE_CODE,ALCOHOL_TEST_CODE,BAC_CODE,CDL_FLAG,CLASS,CONDITION_CODE,DATE_OF_BIRTH,DRUG_TESTRESULT_CODE,DRUG_TEST_CODE,EJECT_CODE,EMS_UNIT_LABEL,EQUIP_PROB_CODE,FAULT_FLAG,INJ_SEVER_CODE,LICENSE_STATE_CODE,MOVEMENT_CODE,OCC_SEAT_POS_CODE,PED_LOCATION_CODE,PED_OBEY_CODE,PED_TYPE_CODE,PED_VISIBLE_CODE,PERSON_ID,PERSON_TYPE,REPORT_NO,SAF_EQUIP_CODE,SEX_CODE,VEHICLE_ID,had_fatality,SEX_CODE2
0,1.0,NaN,0.0,NaN,N,C,0.0,1952-04-20 00:00:00,NaN,0.0,1.0,NaN,1.0,N,No Injury,PA,NaN,NaN,NaN,NaN,NaN,NaN,48dd00ee-e033-47e7-ad1e-0b734020301b,D,AB4284000S,13.0,F,eb6aadb8-dacb-4744-a1a7-ab812c96f27f,0,F
1,1.0,NaN,0.0,NaN,N,NaN,0.0,1985-05-28 00:00:00,NaN,0.0,0.0,NaN,0.0,N,No Injury,MD,NaN,NaN,NaN,NaN,NaN,NaN,166296bd-ffd3-4c16-aa74-4f4bf4139d8d,D,AB4313000X,13.0,F,b463eb20-2f01-4200-9d6f-b18888ce2593,0,F


In [20]:
people['SEX_CODE2'] = people['SEX_CODE'].str.replace(r"F","1")
people['SEX_CODE2'] = people['SEX_CODE'].str.replace(r"M","0")
people['SEX_CODE2'] = people['SEX_CODE2'].str.replace(r"F","1")
people['SEX_CODE2'] = people['SEX_CODE2'].str.replace(r"M","0")

In [21]:
train_df = people[['had_fatality','SEX_CODE2']]

Confirm that your `train_df` has two columns and 815,827 rows.

> **Tip:** If you have more rows, make sure you dropped all of the rows with Unknown sex.
>
> **Tip:** If you have more columns, make sure you only have your input and output columns.

### Run your regression

See the effect of sex on whether the person's injuries are fatal or not. After we train the regression, we can use my ✨favorite technique✨ to display features and their odds ratios:
    
```python
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)
```

In [22]:
train_df.head()

,had_fatality,SEX_CODE2
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [23]:
X = train_df.drop(columns='had_fatality')
y = train_df.had_fatality

clf = LogisticRegression(C=1e9, solver='lbfgs')

clf.fit(X, y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients
}).sort_values(by='coefficient', ascending=False)

,feature,coefficient
0,SEX_CODE2,-0.71481


In [30]:
people.to_csv('people_2.csv')

In [31]:
people_2 = pd.read_csv("people_2.csv")

### Use words to interpret this result

In [ ]:
# sex has not really a big impact on whether an accident was fatal or not 

## Adding more features

The actual crash data has more details - whether it was snowy/icy, whether it was a highway, etc. 

Read in `combined-crash-data.csv`, calling it **`crashes`**, and merge it with our people dataset. I'll save you a lookup: the `REPORT_NO` is what matches between the two.

In [3]:
crashes = pd.read_csv("data/combined-crash-data.csv")

C:\Users\Maximilian\AppData\Local\Programs\Python\Python37-32\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (27,30,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
crashes.shape

(396419, 37)

In [8]:
people_2 = people_2[['had_fatality','SEX_CODE2', 'REPORT_NO']]

In [15]:
people_2.shape

(815827, 3)

In [4]:
merged_df = people_2.merge(crashes, how='left', left_on="REPORT_NO", right_on="REPORT_NO")

In [3]:
# merged_df.to_csv('merged_df.csv')

In [2]:
merged_df = pd.read_csv("merged_df.csv")

C:\Users\Maximilian\AppData\Local\Programs\Python\Python37-32\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6,10,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Examining more possible features

How often was it wet, dry, snowy, icy, etc? **What was the most common surfce condition?**

* **Tip:** We're interested in surface condition, _not_ road condition, _not_ weather condition

In [5]:
merged_df['SURF_COND_CODE2'].value_counts()

2.00     577638
1.00     139282
0.00      27235
4.00       8706
3.00       8697
6.03       1597
99.00      1338
88.00      1019
5.00        425
7.01        252
9.88        136
8.05         36
Name: SURF_COND_CODE2, dtype: int64

In [17]:
merged_df['SURF_COND_CODE2'] = merged_df.SURF_COND_CODE

In [7]:
merged_df.SURF_COND_CODE2.dtype

dtype('O')

In [12]:
# merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"2.00","Dry")
# merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"1.00","Wet")
# merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"0.00","Not Applicable")
# merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"4.00","Ice")
# merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"3.00","Snow")
merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"Slush","Other")
# merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"99.00","Unknown")
# merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"88.00","Other")
# merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"5.00","Mud")
merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"Water","Other")
merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"Oil","Other")
merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].str.replace(r"Sand","Other")

In [19]:
merged_df['SURF_COND_CODE2'] = merged_df['SURF_COND_CODE2'].replace({
    0 : 'Not Applicable',
    1 : 'Wet',
    2 : 'Dry',
    3 : 'Snow',
    4 : 'Ice',
    5 : 'Mud, Dirt, Gravel',
    6.03 : 'Other',
    7.01 : 'Other',
    8.05 : 'Other',
    9.88 : 'Other',
    88 : 'Other',
    99 : 'Unknown'})

In [20]:
merged_df['SURF_COND_CODE2'].value_counts()

Dry                  577638
Wet                  139282
Not Applicable        27235
Ice                    8706
Snow                   8697
Other                  3040
Unknown                1338
Mud, Dirt, Gravel       425
Name: SURF_COND_CODE2, dtype: int64

Do you feel that a **Dry** road condition should be the average of **Wet** and **Snow?**

In [ ]:
#no

The answer to that should be *no*, which means we can't use this data as numeric data. We want a different coefficient for each of these - I want to know the impact of dry, the impact of wet, the impact of snow, all separately.

Start by **replacing each code with a proper description.** I'll even include them here:

* `00` - Not Applicable
* `01`	- Wet
* `02`	- Dry
* `03`	- Snow
* `04`	- Ice
* `05`	- Mud, Dirt, Gravel
* `06`	- Slush
* `07`	- Water (standing/moving)
* `08`	- Sand
* `09`	- Oil
* `88`	- Other
* `99`	- Unknown

But watch out, pandas read the column in as numbers so they might have come through a little differently than their codes.

Confirm you have 147,803 wet, and a few codes you can't understand, like `6.03` and `7.01`.

Replace the codes you don't understand with `Other`.

Confirm you have 3,196 'Other'.

In [ ]:
# i dont but i think i am not wrong... hmm..

## One-hot encoding

We're going to use `pd.get_dummies` to build a variable you'll call `surf_dummies`. Each surface condition should be a `0` or `1` as to whether it was that condition (dry, icy, wet, etc).

Use a `prefix=` so we know they are **surface** conditions.

You'll want to drop the column you'll use as the reference category.

**Before we do this: which column works best as the reference?**

Now build your `surf_dummies` variable.

In [22]:
surf_dummies = pd.get_dummies(merged_df['SURF_COND_CODE2'], prefix='surface').drop('surface_Dry', axis=1)

In [23]:
surf_dummies.head()

,surface_Ice,"surface_Mud, Dirt, Gravel",surface_Not Applicable,surface_Other,surface_Snow,surface_Unknown,surface_Wet
0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0


Confirm your `surf_dummies` looks roughly like this:

|surface_Ice|Surce_Mud, Dirt, Gravel|surface_Not Applicable|...|surface_Wet|
|---|---|---|---|---|
|0|0|0|...|0|
|0|0|0|...|0|
|0|0|1|...|0|
|0|0|1|...|0|
|0|0|0|...|1|

## Another regression

Let's run another regression to see the impact of both **sex and surface condition** on fatalities.

### Build your `train_df`

To build your `train_df`, I recommend doing it either of these two ways. They both first select the important columns, then add in the one-hot encoded `surf_dummies` columns.

```python
train_df = pd.DataFrame({
    'col1': merged.col1,
    'col2': merged.col2,
    'col3': merged.col3,
})
train_df = train_df.join(surf_dummies)
train_df = train_df.dropna()
train_df.head()
```

or like this:

```python
train_df = train_df[['col1','col2','col3']].copy()
train_df = train_df.join(surf_dummies)
train_df = train_df.dropna()
train_df.head()
```

The second one is shorter, but the first one makes it easier to use comments to remove columns later.


In [28]:
train_df = merged_df[['SEX_CODE2','had_fatality']].copy()
train_df = train_df.join(surf_dummies)
train_df = train_df.dropna()
train_df.head()

,SEX_CODE2,had_fatality,surface_Ice,"surface_Mud, Dirt, Gravel",surface_Not Applicable,surface_Other,surface_Snow,surface_Unknown,surface_Wet
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0
2,1,0,0,0,1,0,0,0,0
3,1,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0


### Run your regression and check your odds ratios

Actually no, wait, first - what kind of surface do you think will have the **highest fatality rate?**

In [31]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 815876 entries, 0 to 815875
Data columns (total 9 columns):
SEX_CODE2                    815876 non-null int64
had_fatality                 815876 non-null int64
surface_Ice                  815876 non-null uint8
surface_Mud, Dirt, Gravel    815876 non-null uint8
surface_Not Applicable       815876 non-null uint8
surface_Other                815876 non-null uint8
surface_Snow                 815876 non-null uint8
surface_Unknown              815876 non-null uint8
surface_Wet                  815876 non-null uint8
dtypes: int64(2), uint8(7)
memory usage: 24.1 MB


Confirm your `train_df` has 815,843 rows and 9 columns.

* **Tip:** When you run your regression, if you get an error about not knowing what to do with `U`, it's because you didn't convert your sex to numbers (or if you did, you didn't do it in your original dataframe)

In [32]:
X = train_df.drop(columns='had_fatality')
y = train_df.had_fatality 
clf = LogisticRegression(C=1e9, solver='lbfgs')
clf.fit(X, y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
2,"surface_Mud, Dirt, Gravel",1.119003,3.0618
4,surface_Other,0.596102,1.8150
6,surface_Unknown,0.340243,1.4053
7,surface_Wet,-0.099399,0.9054
1,surface_Ice,-0.558277,0.5722
5,surface_Snow,-0.653307,0.5203
0,SEX_CODE2,-0.716086,0.4887
3,surface_Not Applicable,-0.731252,0.4813


**Is this what you expected?** Why do you think this result might be the case?

In [ ]:
#no

# More features: Vehicles

Maybe the car they're in is related to the car they were in. Luckily, we have this information - **read in `combined_vehicle_data` as `vehicles`.**

In [3]:
vehicles = pd.read_csv("data/combined-vehicle-data.csv")

In [7]:
vehicles.shape

(742133, 35)

## Weights of those cars

The car weights are stored in **another file** since the info had to come from an API. I looked up the VINs - vehicle identification numbers - in a government database to try to get data for each of them.

**Read them and build a new dataframe that is both the vehicle data along with their weights.** You can call it `vehicles` since you don't need the original weightless vehicle data any more.

In [4]:
vaw = pd.read_csv("data/vins_and_weights.csv")

In [5]:
vaw.head()

,VIN,Make,Model,ModelYear,weight
0,2FMDA5143TBB45576,FORD,WINDSTAR,1996,3733.0
1,2G1WC5E37E1120089,CHEVROLET,IMPALA,2014,3618.0
2,5J6RE4H55AL053951,HONDA,CR-V,2010,3389.0
3,1N4AA5AP0EC435185,NISSAN,MAXIMA,2014,3556.0
4,JTHCK262075010440,LEXUS,IS,2007,3527.0


Confirm that your combined `vehicles` dataset should have 534,436 rows and 35 columns. And yes, that's less than we were working with before - you haven't combined it with the people/crashes dataset yet.

In [5]:
vehicles = vehicles.merge(vaw, left_on="VIN_NO", right_on="VIN")
vehicles.shape

(534436, 35)

In [10]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534890 entries, 0 to 534889
Data columns (total 40 columns):
AREA_DAMAGED_CODE1           514889 non-null float64
AREA_DAMAGED_CODE2           313305 non-null float64
AREA_DAMAGED_CODE3           203055 non-null float64
AREA_DAMAGED_CODE_IMP1       530728 non-null float64
AREA_DAMAGED_CODE_MAIN       528765 non-null float64
BODY_TYPE_CODE               534890 non-null float64
COMMERCIAL_FLAG              534890 non-null object
CONTI_DIRECTION_CODE         472477 non-null object
CV_BODY_TYPE_CODE            1323 non-null float64
DAMAGE_CODE                  534890 non-null int64
DRIVERLESS_FLAG              534890 non-null object
FIRE_FLAG                    455860 non-null object
GOING_DIRECTION_CODE         472477 non-null object
GVW_CODE                     1323 non-null float64
HARM_EVENT_CODE              534890 non-null float64
HAZMAT_SPILL_FLAG            337 non-null object
HIT_AND_RUN_FLAG             534890 non-null object
HZM_

### Filter your data

We only want vehicles that are "normal" - somewhere between 1500 and 6000 pounds. Filter your vehicles to only include those in that weight range.

In [7]:
df = vehicles[(vehicles['weight']>= 1500) & (vehicles['weight']<= 6000)]

Confirm that you have 532,370 vehicles in the dataset.

In [19]:
vehicles.columns

Index(['AREA_DAMAGED_CODE1', 'AREA_DAMAGED_CODE2', 'AREA_DAMAGED_CODE3',
       'AREA_DAMAGED_CODE_IMP1', 'AREA_DAMAGED_CODE_MAIN', 'BODY_TYPE_CODE',
       'COMMERCIAL_FLAG', 'CONTI_DIRECTION_CODE', 'CV_BODY_TYPE_CODE',
       'DAMAGE_CODE', 'DRIVERLESS_FLAG', 'FIRE_FLAG', 'GOING_DIRECTION_CODE',
       'GVW_CODE', 'HARM_EVENT_CODE', 'HAZMAT_SPILL_FLAG', 'HIT_AND_RUN_FLAG',
       'HZM_NUM', 'MOVEMENT_CODE', 'NUM_AXLES', 'PARKED_FLAG', 'REPORT_NO',
       'SPEED_LIMIT', 'TOWED_AWAY_FLAG', 'TOWED_VEHICLE_CONFIG_CODE',
       'VEHICLE_ID', 'VEH_MAKE', 'VEH_MODEL', 'VEH_YEAR', 'VIN_NO', 'VIN_x',
       'Make_x', 'Model_x', 'ModelYear_x'],
      dtype='object')

In [9]:
df.to_csv('df_veh.csv')

In [3]:
df_veh = pd.read_csv("df_veh.csv")

C:\Users\Maximilian\AppData\Local\Programs\Python\Python37-32\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
xx = merged_df[['had_fatality', 'VEHICLE_ID']]

## Add this vehicle information to your merged data

Now we'll have a dataframe that contains information on:

* The people themselves and their injuries
* The crash
* The vehicles

Every person came with a `VEHICLE_ID` column that is the vehicle they were in. You'll want to merge on that.

In [2]:
xx = pd.read_csv("xx.csv")

In [7]:
xx.to_csv('xx.csv')
#I lost 3 hrs with memory errors. hence this.

In [4]:
merged2 = pd.merge(xx, df_veh, on='VEHICLE_ID')

Confirm you have 99 columns and 616,212 rows. **That is a lot of possible features!**

## Another regression, because we can't get enough

Build another `train_df` and run another regression about **how car weight impacts the chance of fatalities**. You'll want to confirm that your dataset has 616,212 and 2 columns.

In [8]:
train_df = merged2.copy()
train_df = train_df[['had_fatality', 'weight']]

In [9]:
train_df.shape

(596148, 2)

In [10]:
X = train_df.drop(columns=['had_fatality'])
y = train_df.had_fatality

In [11]:
clf = LogisticRegression(C=1e9, solver='lbfgs')
clf.fit(X, y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
0,weight,-0.000146,0.999854


**Can you translate that into plain English?** Remember weight is in **pounds**.

I feel like pounds isn't the best measure for something like this. Remember how we had to adjust percentages with AP and life expecntancy, and then change around the way we said things? It sounded like this:

> Every 10% increase in unemployment translates to a year and a half loss of life expectancy

Instead of every single pound, maybe we could do every... some other number of pounds? One hundred? One thousand?

**Run another regression with weight in thousands of pounds.** Get another odds ratio. Give me another sentence English.

In [13]:
train_df['weight'] = train_df.weight/1000

In [15]:
train_df = merged2.copy()
train_df = train_df[['had_fatality', 'weight']]
train_df['weight'] = train_df.weight/1000

In [18]:
X = train_df.drop(columns=['had_fatality'])
y = train_df.had_fatality

In [19]:
clf = LogisticRegression(C=1e9, solver='lbfgs')
clf.fit(X, y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
0,weight,-0.146277,0.863918


In [ ]:
# Every thousand pounds heavier your car is increase translates to a 15% decrease in fatalities

# Two-car accidents, struck and striker

Here's the thing, though: **it isn't just the weight of your car.** It's the weight of both cars! If I'm in a big car and I have a wreck with a smaller car, it's the smaller car that's in trouble.

To get that value, we need to do some **feature engineering**, some calculating of *new* variables from our *existing* variables.

We need to jump through some hoops to do that.

##  Two-car accidents

First we're going to count how many vehicles were in each accident. Since we're looking to compare the weight of two cars hitting each other, **we're only going to want crashes with only two cars.**

In [22]:
counted = df_veh.REPORT_NO.value_counts()
counted.head(10)

CB53480003      28
MDTA1229000H    26
AS0286000B      14
CE4636002C      14
MSP6063006C     13
MSP67460095     13
AE5607002W      12
CC0237003W      12
MSP63070049     12
MSP6297005S     11
Name: REPORT_NO, dtype: int64

By using `.value_counts` I can see how many cars were in each crash, and now I'm going to filter to get a list of all of the ones with two vehicles.

In [23]:
two_car_report_nos = counted[counted == 2].index
two_car_report_nos

Index(['DA4118000K', 'DE3107000H', 'MCP28480055', 'AE4727002T', 'ADJ252002H',
       'ADJ592001P', 'MSP6049009H', 'MSP6852002J', 'MSP6278003L', 'DA33630002',
       ...
       'AE56260005', 'AE3788003T', 'BS05140002', 'DA3935001Z', 'ZU6930000X',
       'DA39890007', 'ZJ0499000B', 'MDTA16470014', 'ADF880001K',
       'MSP58610047'],
      dtype='object', length=137214)

And now we'll filter my vehicles so we only have those that were in two-vehicle crashes.

In [25]:
vehicles = df_veh[df_veh.REPORT_NO.isin(two_car_report_nos)]

### Struck and striker

To do the math correctly, we need both the risk of someone dying in the smaller car _and_ the risk of someone dying in the bigger car. To do this we need to separate our cars into two groups:

* The 'struck' vehicle: did the person die inside?
* The 'striker' vehicle: how much heavier was it than the struck car?

But we don't know which car was which, so we have to try out both versions - pretending car A was the striker, then pretending car B was the striker. It's hard to explain, but you can read `Pounds That Kill - The External Costs of Vehicle Weight.pdf` for more details on how it works.

In [26]:
cars_1 = vehicles.drop_duplicates(subset='REPORT_NO', keep='first')
cars_2 = vehicles.drop_duplicates(subset='REPORT_NO', keep='last')

In [27]:
cars_merged_1 = cars_1.merge(cars_2, on='REPORT_NO', suffixes=['_striker', '_struck'])
cars_merged_2 = cars_2.merge(cars_1, on='REPORT_NO', suffixes=['_striker', '_struck'])
vehicles_complete = pd.concat([cars_merged_1, cars_merged_2])
vehicles_complete.head()

,Unnamed: 0_striker,AREA_DAMAGED_CODE1_striker,AREA_DAMAGED_CODE2_striker,AREA_DAMAGED_CODE3_striker,AREA_DAMAGED_CODE_IMP1_striker,AREA_DAMAGED_CODE_MAIN_striker,BODY_TYPE_CODE_striker,COMMERCIAL_FLAG_striker,CONTI_DIRECTION_CODE_striker,CV_BODY_TYPE_CODE_striker,DAMAGE_CODE_striker,DRIVERLESS_FLAG_striker,FIRE_FLAG_striker,GOING_DIRECTION_CODE_striker,GVW_CODE_striker,HARM_EVENT_CODE_striker,HAZMAT_SPILL_FLAG_striker,HIT_AND_RUN_FLAG_striker,HZM_NUM_striker,MOVEMENT_CODE_striker,NUM_AXLES_striker,PARKED_FLAG_striker,REPORT_NO,SPEED_LIMIT_striker,TOWED_AWAY_FLAG_striker,TOWED_VEHICLE_CONFIG_CODE_striker,VEHICLE_ID_striker,VEH_MAKE_striker,VEH_MODEL_striker,VEH_YEAR_striker,VIN_NO_striker,VIN_striker,Make_striker,Model_striker,ModelYear_striker,weight_striker,Unnamed: 0_struck,AREA_DAMAGED_CODE1_struck,AREA_DAMAGED_CODE2_struck,AREA_DAMAGED_CODE3_struck,AREA_DAMAGED_CODE_IMP1_struck,AREA_DAMAGED_CODE_MAIN_struck,BODY_TYPE_CODE_struck,COMMERCIAL_FLAG_struck,CONTI_DIRECTION_CODE_struck,CV_BODY_TYPE_CODE_struck,DAMAGE_CODE_struck,DRIVERLESS_FLAG_struck,FIRE_FLAG_struck,GOING_DIRECTION_CODE_struck,GVW_CODE_struck,HARM_EVENT_CODE_struck,HAZMAT_SPILL_FLAG_struck,HIT_AND_RUN_FLAG_struck,HZM_NUM_struck,MOVEMENT_CODE_struck,NUM_AXLES_struck,PARKED_FLAG_struck,SPEED_LIMIT_struck,TOWED_AWAY_FLAG_struck,TOWED_VEHICLE_CONFIG_CODE_struck,VEHICLE_ID_struck,VEH_MAKE_struck,VEH_MODEL_struck,VEH_YEAR_struck,VIN_NO_struck,VIN_struck,Make_struck,Model_struck,ModelYear_struck,weight_struck
0,2,6.0,7.0,NaN,6.0,6.0,20.0,N,N,NaN,3,N,N,N,NaN,1.0,NaN,N,NaN,6.0,NaN,N,CB5190006B,55,N,0,0003b659-2785-4868-8877-0b786a284827,TOYT,TK,2011.0,5TFUY5F1XBX167340,5TFUY5F1XBX167340,TOYOTA,TUNDRA,2011,5480.0,2528,3.0,5.0,6.0,6.0,6.0,23.08,N,N,NaN,5,N,N,N,NaN,1.0,NaN,N,NaN,6.0,NaN,N,55,Y,0,0e7da302-5f53-45c8-8b28-393cdc38171d,JEEP,CHEROKEE,2015.0,1C4RJFBGXFC751955,1C4RJFBGXFC751955,JEEP,GRAND CHEROKEE,2015,4545.0
1,3,8.0,NaN,NaN,8.0,8.0,2.0,N,W,NaN,2,N,NaN,W,NaN,1.0,NaN,N,NaN,8.0,NaN,N,ADJ4590035,5,N,0,00050484-d08f-4b6e-bc7e-9ec270e94660,HONDA,CIVIC,2015.0,2HGFG4A59FH702545,2HGFG4A59FH702545,HONDA,CIVIC,2015,2754.0,10723,99.0,NaN,NaN,99.0,99.0,2.00,N,W,NaN,99,N,N,W,NaN,1.0,NaN,Y,NaN,1.0,NaN,N,5,N,0,3ebfe7fc-4832-414d-a5e8-391b65a2f280,HONDA,ACCORD,2013.0,1HGCT2B81DA005695,1HGCT2B81DA005695,HONDA,ACCORD,2013,3163.0
2,4,12.0,NaN,NaN,12.0,12.0,2.0,N,N,NaN,4,N,NaN,N,NaN,0.0,NaN,N,NaN,1.0,NaN,N,ADJ849000Z,10,N,0,00057af4-d848-4cee-b854-707f57581f4e,HONDA,ACCORD,2003.0,1HGCM66313A037175,1HGCM66313A037175,HONDA,ACCORD,2003,3023.0,13102,12.0,NaN,NaN,12.0,12.0,2.00,N,NaN,NaN,3,Y,NaN,NaN,NaN,1.0,NaN,N,NaN,10.0,NaN,Y,0,N,0,4cf496cc-f537-447e-bb8e-3ef48c8c3ddf,HONDA,TK,2010.0,5J6RE3H73AL008454,5J6RE3H73AL008454,HONDA,CR-V,2010,3389.0
3,5,1.0,11.0,12.0,12.0,12.0,2.0,N,S,NaN,4,N,N,S,NaN,1.0,NaN,N,NaN,12.0,NaN,N,AE5207008Z,30,Y,0,00089d4a-7038-4693-9e02-b402676631af,FORD,4D,2016.0,1FADP3K28GL258987,1FADP3K28GL258987,FORD,FOCUS,2016,2932.0,5235,11.0,12.0,NaN,12.0,12.0,23.08,N,N,NaN,4,N,N,N,NaN,1.0,NaN,N,NaN,1.0,NaN,N,30,Y,0,1e69b284-719f-429b-baae-6ae4976a546b,BUICK,TK,2002.0,3G5DB03E52S528316,3G5DB03E52S528316,BUICK,RENDEZVOUS,2002,4024.0
4,6,2.0,NaN,NaN,2.0,2.0,2.0,N,S,NaN,99,N,N,S,NaN,1.0,NaN,Y,NaN,1.0,NaN,N,ADJ859000Y,15,N,0,0010076b-0a45-45f3-8796-f387b39cd85d,HONDA,CIVIC,2009.0,2HGFA16689H357624,2HGFA16689H357624,HONDA,CIVIC,2009,2678.0,29827,10.0,NaN,NaN,10.0,10.0,2.00,N,NaN,NaN,3,Y,N,NaN,NaN,1.0,NaN,N,NaN,10.0,NaN,Y,15,N,0,b14273cb-b3e5-42c7-af59-df069e8c2f0f,HONDA,CIVIC,2011.0,2HGFG1B87BH517469,2HGFG1B87BH517469,HONDA,CIVIC,2011,2703.0


## Put people in their cars

Which car was each person in? We'll assign that now.

In [32]:
merged = people.merge(vehicles_complete, left_on='VEHICLE_ID', right_on='VEHICLE_ID_struck')
merged.head(3)

# Add the crash details

You did this already! I'm going to do it for you. We're merging on `REPORT_NO_x` because there are so many `REPORT_NO` columns duplicated across our files that pandas started giving them weird names.

In [ ]:
merged = merged.merge(crashes, left_on='REPORT_NO_x', right_on='REPORT_NO')
merged.head(3)

## Filter

We already filtered out vehicles by weight, so we don't have to do that again.

# Calculated features

I'm sure you forgot what all the features are, so we'll bring back whether there was a fatality or not

## Feature: Accident was fatal

In [ ]:
merged['had_fatality'] = (merged.INJ_SEVER_CODE == 5).astype(int)
merged.had_fatality.value_counts()

## Feature: Weight difference

**Remove everything missing weights for strikers or struck vehicles.** You might need to `merged.columns` to remind yourself what the column names are.

In [ ]:
merged = merged.dropna(subset=['weight_striker', 'weight_struck'])

Confirm your dataset has roughly 335,000 rows.

In [ ]:
merged.shape

Create a new feature called `weight_diff` about how much heavier the striking car was compared to the struck car. **Make sure you've done the math correctly!**

In [ ]:
merged['weight_diff'] = merged.weight_striker - merged.weight_struck

### Feature adjustment

Make all of your weight columns in **thousands of pounds** instead of just in pounds. It'll help you interpret your results much better.

In [ ]:
merged['weight_diff'] = merged['weight_diff'] / 1000
merged['weight_struck'] = merged['weight_struck'] / 1000
merged['weight_striker'] = merged['weight_striker'] / 1000

# Another regression!!!

**What is the impact of weight difference on fatality rate?** Create your `train_df`, drop missing values, run your regression, analyze your odds ratios.

In [ ]:
train_df = merged[['had_fatality', 'weight_diff']]
X = train_df.drop(columns=['had_fatality'])
y = train_df.had_fatality
logit = LogisticRegression()
logit.fit(X, y)

In [ ]:
feature_names = X.columns
coefficients = clf.coef_[0]
pd.DataFrame({
   'feature': feature_names,
   'coefficient (log odds ratio)': coefficients,
   'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

Please translate your odds ratio into plain English.

## Adding in more features

How about speed limit? That's important, right? We can add the speed limit of the striking vehicle with `SPEED_LIMIT_striker`.

In [ ]:
train_df = merged[['had_fatality', 'weight_diff', 'SPEED_LIMIT_striker']]

In [ ]:
X = train_df.drop(columns=['had_fatality'])
y = train_df.is_fatality
logit = LogisticRegression()
logit.fit(X, y)

In [ ]:
feature_names = X.columns
coefficients = logit.coef_[0]
pd.DataFrame({
   'feature': feature_names,
   'coefficient (log odds ratio)': coefficients,
   'odds ratio': np.exp(coefficients)
}).sort_values(by='odds ratio', ascending=False)

Can you translate the speed limit odds ratio into plain English?

### Feature engineering: Speed limits

Honestly, that's a pretty bad way to go about things. What's more fun is if we **translate speed limits into bins.**

First, we'll use `pd.cut` to assign each speed limit a category.

In [ ]:
speed_bins = [-np.inf, 10, 20, 30, 40, 50, np.inf]
merged['speed_bin'] = pd.cut(merged.SPEED_LIMIT_struck, bins=speed_bins)
merged[['SPEED_LIMIT_striker', 'speed_bin']].head(10)

Then we'll one-hot encode around 20-30mph speed limits.

In [ ]:
speed_dummies = pd.get_dummies(merged.speed_bin, 
                               prefix='speed').drop('speed_(20.0, 30.0]', axis=1)
speed_dummies.head()

## Running a regression

I like this layout for creating `train_df`, it allows us to easily add dummies and do a little replacing/encoding when we're building binary features like for sex.

> If the below gives you an error, it's because `SEX_CODE` is already a number. In that case, just remove `.replace({'M': 1, 'F': 0, 'U': np.nan })`.

In [ ]:
# Start with our normal features
train_df = pd.DataFrame({
    'weight_diff': merged.weight_diff,
    'sex': merged.SEX_CODE,#.replace({'M': 1, 'F': 0, 'U': np.nan }),
    'had_fatality': merged.had_fatality,
})
# Add the one-hot encoded features
train_df = train_df.join(speed_dummies)
train_df = train_df.join(surf_dummies)
# Drop missing values
train_df = train_df.dropna()
train_df.head()

Describe the impact of the different variables in simple language. What has the largest impact?

## Now you pick the features

Up above you have examples of:

* Creating features from numbers (speed limits)
* Creating features from 0/1 (sex)
* Creating features from binning numbers that are one-hot encoded (speed limit bins - `speed_bins`)
* Creating features from categories that are one-hot encoded (surface - `surf_dummies`

What else do you think matters? Try to plug in more features and see if you can get anything interesting.

> * **Hot tip:** The funniest/most interesting thing feature you can add is also the dumbest. Ask me about it in #algorithms if you end up getting down here.